<a href="https://colab.research.google.com/github/rickygrosvenor-pramanick/learn-ml/blob/main/pytorch/nn_workflow_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Create Dataset
from sklearn.datasets import make_circles

X, y = make_circles(n_samples=1000,
                    noise=0.7,
                    factor=0.8,
                    random_state=42)

In [3]:
# Convert to Tensors
import torch

X = torch.from_numpy(X).type(torch.float)
y = torch.from_numpy(y).type(torch.float)

In [4]:
# Train-Test Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Agnostic Code
import torch
from torch import nn

device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
# Investigate Data Format
# X gives 2 numbers as input, y returns 1 number as output
X_train[:5], y_train[:5]

(tensor([[ 1.3037,  0.1872],
         [-0.3370, -0.6710],
         [-1.4545,  0.4610],
         [-0.7890, -0.2558],
         [-0.0992,  1.1932]]),
 tensor([1., 0., 0., 0., 1.]))

In [7]:
# Create Model Class
class CircleModelV1(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer_1 = nn.Linear(in_features=2, out_features=8)
    self.layer_2 = nn.Linear(in_features=8, out_features=3)
    self.layer_3 = nn.Linear(in_features=3, out_features=1)

  def forward(self, x):
    return self.layer_3(self.layer_2(self.layer_1(x)))

In [8]:
# Create Model Instance
model1 = CircleModelV1().to(device)
model1

CircleModelV1(
  (layer_1): Linear(in_features=2, out_features=8, bias=True)
  (layer_2): Linear(in_features=8, out_features=3, bias=True)
  (layer_3): Linear(in_features=3, out_features=1, bias=True)
)

In [28]:
# Initial Predictions without Training
import math
untrained_preds = model1(X_test.to(device))
untrained_preds_to_int = untrained_preds.round()
untrained_preds_to_int[:5], y_test[:5]

(tensor([[0.],
         [1.],
         [0.],
         [1.],
         [-0.]], grad_fn=<SliceBackward0>),
 tensor([1., 0., 1., 0., 1.]))

In [ ]:
# Create a loss function
